In [1]:
%load_ext autoreload
%autoreload 2

import os
import json
import copy
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
import ray_results_interpreter as rri
import subprocess
import concurrent.futures
from main_run import MainRun

/user/ml4723/.conda/envs/neural_inventory_control/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/user/ml4723/.conda/envs/neural_inventory_control/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [16]:
lower_bounds_path = "/user/ml4723/Prj/NIC/ray_results/finals_transshipment_HDPO/lower_bounds.csv"
lower_bounds_df = pd.read_csv(lower_bounds_path)
print(lower_bounds_df.to_string())

    n_stores  store_lead_time  store_underage_cost  stores_correlation  lower_bound
0          3                2                    4                 0.0     5.753735
1          3                2                    4                 0.5     5.753735
2          3                2                    9                 0.0     7.213631
3          3                2                    9                 0.5     7.213631
4          3                6                    4                 0.0     7.717182
5          3                6                    4                 0.5     7.717182
6          3                6                    9                 0.0     9.675263
7          3                6                    9                 0.5     9.675263
8          5                2                    4                 0.0     5.296388
9          5                2                    4                 0.5     5.296388
10         5                2                    9                 0.0     6

In [6]:
# Create vanilla results dataframe for finals_transshipment_HDPO setting
testset_name = "finals_transshipment_HDPO"

results_interpretor = rri.RayResultsinterpreter()

def custom_data_filler(out_row, reference_row):
    out_row['path'] = reference_row['path']

def default_condition_setter(condition_name):
    return None

architectures = {
    "Vanilla NN": lambda config: f'/user/ml4723/Prj/NIC/ray_results/{testset_name}/vanilla_transshipment',
}

sort_by = 'dev_loss'
pick_row_from_run_by = 'dev_loss'

config = "transshipment_backlogged"
n_stores_list = [3, 5, 10]
store_lead_times = [2, 6]
store_underage_costs = [4, 9]
stores_correlation_list = [0.0, 0.5]

dfs = []

for arch_name, path_fn in architectures.items():
    path = path_fn(config)

    df = results_interpretor.make_table(
        {1: path},
        {
            'n_stores': n_stores_list,
            'store_underage_cost': store_underage_costs,
            'store_lead_time': store_lead_times,
            'stores_correlation': stores_correlation_list,
        },
        default_condition_setter,
        custom_data_filler,
        sort_by=sort_by,
        pick_row_from_run_by=pick_row_from_run_by,
        test_loss_limit=25
    )
    if df.empty:
        continue

    df.insert(2, 'Architecture Class', arch_name)
    df.insert(1, 'hyperparam_name', arch_name)
    df['config'] = config
    dfs.append(df)

if not dfs:
    raise ValueError("No dataframes found for the given settings.")

df = pd.concat(dfs, ignore_index=True)

In [8]:
import numpy as np

test_loss_column = 'Test Loss'
test_loss_filename = 'transshipment_backlogged_test_loss.txt'

test_losses = []
for _, row in df.iterrows():
    test_loss_path = str(row['path']) + "/" + test_loss_filename
    try:
        with open(test_loss_path, 'r') as f:
            value = float(f.read().strip())
    except Exception:
        value = np.nan
    test_losses.append(value)

df[test_loss_column] = test_losses

In [17]:
import numpy as np
import pandas as pd
import os

lead_time_col = "store_lead_time"
underage_cost_col = "store_underage_cost"
n_stores_col = "n_stores"
correlation_col = "stores_correlation"
dev_loss_col = "Dev Loss"
train_loss_col = "Train Loss"
test_loss_col = "Test Loss"
lower_bound_col = "Lower bound"
hyperparam_col = "hyperparam_name"

# lower_bounds_transshipment_df is assumed to be provided and contains the columns:
# n_stores, store_lead_time, store_underage_cost, stores_correlation, lower_bound

# Set index for fast lookup
lower_bounds_transshipment_df = lower_bounds_df.set_index(
    ["n_stores", "store_lead_time", "store_underage_cost", "stores_correlation"]
)

rows = []
for idx, lb_row in lower_bounds_transshipment_df.iterrows():
    n_stores, lead_time, underage_cost, correlation = idx
    lower_bound = lb_row["lower_bound"]

    # Find the matching row in df
    match = df[
        (df[n_stores_col] == n_stores) &
        (df[lead_time_col] == lead_time) &
        (df[underage_cost_col] == underage_cost) &
        (df[correlation_col] == correlation)
    ]
    if match.empty:
        continue
    # If multiple, just take the first
    row = match.iloc[0]

    train_loss = row.get(train_loss_col, np.nan)
    dev_loss = row.get(dev_loss_col, np.nan)
    test_loss = row.get(test_loss_col, np.nan)
    if np.isnan(test_loss) or np.isnan(lower_bound):
        continue

    test_gap = 100 * (test_loss - lower_bound) / lower_bound
    test_gap_str = f"{test_gap:.2f}"

    progress_path = os.path.join(row["path"], "progress.csv")
    grad_steps_to_1pct = ""
    time_to_1pct = ""
    try:
        progress_df = pd.read_csv(progress_path)
        dev_gap_threshold = 0.01 * lower_bound
        first_time = progress_df["time_total_s"].iloc[0]
        first_step = progress_df["training_iteration"].iloc[0]
        found = False
        for _, prog_row in progress_df.iterrows():
            prog_dev_gap = abs(prog_row["dev_loss"] - lower_bound)
            if prog_dev_gap <= dev_gap_threshold:
                grad_steps = prog_row["training_iteration"] - first_step
                time_to_1pct = prog_row["time_total_s"] - first_time
                grad_steps_scaled = grad_steps * 10 * 32  # 10x for epoch, 32x for batch size 1024
                grad_steps_to_1pct = int(grad_steps_scaled)
                time_to_1pct = int(time_to_1pct)
                found = True
                break
        if not found:
            grad_steps_to_1pct = ""
            time_to_1pct = ""
    except Exception:
        grad_steps_to_1pct = ""
        time_to_1pct = ""

    rows.append([
        n_stores,
        lead_time,
        underage_cost,
        correlation,
        round(lower_bound, 2) if not np.isnan(lower_bound) else "",
        round(train_loss, 2) if not np.isnan(train_loss) else "",
        round(dev_loss, 2) if not np.isnan(dev_loss) else "",
        round(test_loss, 2) if not np.isnan(test_loss) else "",
        test_gap_str,
        grad_steps_to_1pct,
        time_to_1pct
    ])

table11_df = pd.DataFrame(rows, columns=[
    "Number of stores",
    "Store leadtime",
    "Store underage cost",
    "Pairwise correlation",
    "Lower bound",
    "Train loss",
    "Dev loss",
    "Test loss",
    "Test gap (%)",
    "Gradient steps to 1% dev gap",
    "Time to 1% dev gap (s)"
])

sort_cols = ["Number of stores", "Store leadtime", "Store underage cost", "Pairwise correlation"]
table11_df = table11_df.sort_values(sort_cols).reset_index(drop=True)
table11_df

,Number of stores,Store leadtime,Store underage cost,Pairwise correlation,Lower bound,Train loss,Dev loss,Test loss,Test gap (%),Gradient steps to 1% dev gap,Time to 1% dev gap (s)
0,3,2,4,0.0,5.75,5.15,5.15,5.15,-10.45,,
1,3,2,4,0.5,5.75,5.77,5.75,5.75,0.00,2880,465
2,3,2,9,0.0,7.21,6.45,6.45,6.46,-10.45,,
3,3,6,4,0.0,7.72,7.27,7.27,7.29,-5.57,960,148
4,3,6,4,0.5,7.72,7.74,7.72,7.73,0.18,2560,412
5,3,6,9,0.0,9.68,9.11,9.12,9.13,-5.61,960,150
6,5,2,4,0.0,5.30,4.59,4.59,4.60,-13.19,,
7,5,2,9,0.0,6.64,5.76,5.76,5.76,-13.19,,
8,5,6,4,0.0,7.17,6.67,6.67,6.68,-6.88,1280,215
9,5,6,4,0.5,7.17,7.19,7.17,7.19,0.24,5760,820


In [7]:
mode = "test"
setting_names = ['transshipment_backlogged']

models = []
for _, row in df.iterrows():
    models.append(str(row['path']) + '/model.pt')

gpus = [0, 1, 2, 3]

import nest_asyncio
import asyncio

nest_asyncio.apply()

async def run_main_run(model_path, setting_name, gpu_idx, semaphore):
    async with semaphore:
        try:
            hyperparam_name = model_path.split('/')[7]
            print(f"Running main_run.py for path {model_path}")
            cmd = [
                "/user/ml4723/.conda/envs/neural_inventory_control/bin/python",
                "main_run.py",
                mode,
                setting_name,
                hyperparam_name,
                str(model_path),
                str(gpus[gpu_idx])
            ]
            env = {
                **os.environ,
                "MKL_THREADING_LAYER": "GNU",
                "MKL_SERVICE_FORCE_INTEL": "1"
            }
            process = await asyncio.create_subprocess_exec(
                *cmd,
                env=env,
                cwd="/user/ml4723/Prj/NIC/",
                stdout=asyncio.subprocess.PIPE,
                stderr=asyncio.subprocess.PIPE
            )
            stdout, stderr = await process.communicate()
            if process.returncode != 0:
                print(f"Error running main_run.py for path {model_path}: {stderr.decode()}")
        except Exception as e:
            print(f"Unexpected error running main_run.py for path {model_path}: {e}")

async def main():
    max_concurrent = 6 * len(gpus)
    semaphore = asyncio.Semaphore(max_concurrent)
    tasks = []
    gpu_idx = 0
    for setting_name in setting_names:
        for path in models:
            tasks.append(run_main_run(path, setting_name, gpu_idx, semaphore))
            gpu_idx = (gpu_idx + 1) % len(gpus)
    await asyncio.gather(*tasks)

await main()


Running main_run.py for path /user/ml4723/Prj/NIC/ray_results/finals_transshipment_HDPO/vanilla_transshipment/run_2025-05-23_19-22-44/run_d489d_00000_0_config=transshipment_backlogged,early_stop_check_epochs=10,n_stores=3,repeats=1,stop_if_no_improve_for_epochs=50_2025-05-23_19-22-45/model.pt
Running main_run.py for path /user/ml4723/Prj/NIC/ray_results/finals_transshipment_HDPO/vanilla_transshipment/run_2025-05-23_19-22-44/run_d489d_00039_39_config=transshipment_backlogged,early_stop_check_epochs=10,n_stores=3,repeats=2,stop_if_no_improve_for_epochs=5_2025-05-23_19-22-46/model.pt
Running main_run.py for path /user/ml4723/Prj/NIC/ray_results/finals_transshipment_HDPO/vanilla_transshipment/run_2025-05-23_19-22-44/run_d489d_00012_12_config=transshipment_backlogged,early_stop_check_epochs=10,n_stores=3,repeats=2,stop_if_no_improve_for_epochs=5_2025-05-23_19-22-46/model.pt
Running main_run.py for path /user/ml4723/Prj/NIC/ray_results/finals_transshipment_HDPO/vanilla_transshipment/run_2025